<a href="https://colab.research.google.com/github/K-Domar/MyRepo/blob/main/Homework4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [6]:
hea = pd.read_excel('https://raw.githubusercontent.com/K-Domar/MyRepo/main/dataset.xlsx')
hea.head()

,Alloy,No,Component,Fraction,a,delta,Tm,D_Tm,Hmix,σHmix,Sid,Elec_nega,D_elec_nega,VEC,d_VEC,BulkModulus,D_Bulk,Phase,Phase_inshort,AM？,IM?,SS?,Phase_3,Ref,From,Size,Method
0,AlB12,2,Al B,1 12,0.867077,0.188079,2239.190000,376.928897,0.000000,0.000000,0.271189,2.006923,0.114582,3.000000,0.000000,3.012308e+11,65.018523,IM,IM,0,1,0,2.0,NaN,三元相图,NaN,NaN
1,B4Co,2,B Co,4 1,0.906200,0.190245,2232.000000,232.000000,-15.360000,3.456000,0.500402,2.008000,0.064000,4.200000,2.400000,2.920000e+11,56.000000,IM,IM,0,1,0,2.0,NaN,三元相图,NaN,NaN
2,AlB2,2,Al B,1 2,1.024000,0.281738,1876.490000,666.815837,0.000000,0.000000,0.636514,1.896667,0.202704,3.000000,0.000000,2.386667e+11,115.022703,IM,IM,0,1,0,2.0,NaN,三元相图,NaN,NaN
3,B6Co2Nb2,3,B Co Nb,6 2 2,1.028000,0.253786,2312.400000,313.581632,-41.440000,8.138784,0.950271,1.920000,0.171581,4.600000,2.332381,2.620000e+11,71.105555,IM,IM,0,1,0,2.0,NaN,三元相图,NaN,NaN
4,HfB2,2,Hf B,1 2,1.072667,0.333118,2400.666667,74.481914,-58.666667,3.456966,0.636514,1.793333,0.348839,3.333333,0.471405,2.500000e+11,98.994949,IM,IM,0,1,0,2.0,NaN,三元相图,NaN,NaN
